In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

def conv2D_batchnorm(*args, **kwargs):
    return keras.Sequential([
        layers.Conv2D(*args, **kwargs),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

def basic_block(channel_out):
    return keras.Sequential([
        conv2D_batchnorm(
            channel_out//4, 1, strides=1, padding="same"
        ),
        conv2D_batchnorm(
            channel_out//4, (3, 1), strides=1, padding="same"
        ),
        conv2D_batchnorm(
            channel_out//4, (1, 3), strides=1, padding="same"
        ),
        conv2D_batchnorm(
            channel_out//4, 1, strides=1, padding="same"
        ),
    ])


In [10]:
class global_context(layers.Layer):
    def __init__(self, ksize, strides):
        super().__init__()
        self.ksize = ksize
        self.strides = strides
    
    def call(self, channel_in):
        x = layers.AveragePooling2D(
            pool_size=self.ksize, strides=self.strides, padding="same"
        )(channel_in)
        cx = layers.Lambda(lambda e: tf.math.square(e))(x)
        cx = layers.Lambda(lambda e: tf.math.reduce_mean(e))(cx)
        return layers.Lambda(
            lambda e: tf.math.divide(e[0], e[1])
        )([x, cx])

def lprnet(
    n_classes,
    shape=(24, 94, 3),
):
    input_layer = layers.Input(shape)
    x = conv2D_batchnorm(
        64, (3, 3), strides=1, padding="same"
    )(input_layer)
    x = layers.MaxPool2D(pool_size=(3, 3), strides=1)(x)
    x2 = basic_block(128)(x)
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(1, 2))(x2)
    x = basic_block(256)(x)
    x3 = basic_block(256)(x)
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(1, 2))(x3)
    x = layers.Dropout(0.5)(x)
    x = conv2D_batchnorm(
        256, (4, 1), strides=1, padding="same"
    )(x)
    x = layers.Dropout(0.5)(x)
    x = conv2D_batchnorm(
        n_classes, (1, 13), strides=1, padding="same"
    )(x)

    """
    # Global Context
    cx = layers.Lambda(lambda e: tf.math.square(e))(x)
    cx = layers.Lambda(lambda e: tf.math.reduce_mean(e))(cx)
    x0  = layers.Lambda(
        lambda e: tf.math.divide(e[0], e[1])
    )([x, cx])

    x1 = global_context(
        ksize=[1, 4],
        strides=[1, 4],
    )(input_layer)
    x2 = global_context(
        ksize=[1, 4],
        strides=[1, 4],
    )(x2)
    x3 = global_context(
        ksize=[1, 2],
        strides=[1, 2],
    )(x3)

    x = layers.Lambda(
        lambda e: tf.concat([e[0], e[1], e[2], e[3]], 3)
    )([x1, x2, x3, x0])
    x = layers.Conv2D(
        n_classes,
        kernel_size=(1,1),
        strides=(1,1),
    )(x)
    logits = layers.Lambda(
        lambda x: tf.math.reduce_mean(x, axis=1)
    )(x)
    output_layer = layers.Softmax()(logits)
    """


    return keras.Model(input_layer, x)

In [11]:
CHAR_OUTPUTS = "ABCDEFGHJKLMNPQRSTUVWXYZ0123456789"
CHAR_ENCODE = {c:i for i, c in enumerate(CHAR_OUTPUTS)}
N_OUTPUTS = len(CHAR_OUTPUTS) + 1
print(f"N_OUTPUTS: {N_OUTPUTS}")

model = lprnet(N_OUTPUTS)
learning_rate_scheduler = keras.optimizers.schedules.ExponentialDecay(
    1e-3, 
    decay_steps=500,
    decay_rate=0.995,
    staircase=True
)
model.compile(
    optimizer=keras.optimizers.Adam(
        learning_rate=learning_rate_scheduler
    ),
    loss=ctc_loss
)

N_OUTPUTS: 35


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 24, 94, 3)]       0         
                                                                 
 sequential_72 (Sequential)  (None, 24, 94, 64)        2048      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 22, 92, 64)       0         
 g2D)                                                            
                                                                 
 sequential_77 (Sequential)  (None, 22, 92, 32)        9856      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 20, 45, 32)       0         
 g2D)                                                            
                                                                 
 sequential_82 (Sequential)  (None, 20, 45, 64)        32000 

In [ ]:
test_input = tf.random.uniform((10, 24, 94, 3))
test_output = model.predict(test_input)

In [ ]:
test_output.shape

In [ ]:
from generators.plates_generator import ImageGenerator

generator = ImageGenerator()
data, labels = generator.generate_images(1000)

In [ ]:
from matplotlib import pyplot as plt

plt.axis(False)
plt.imshow(data[0], interpolation='nearest')
plt.show()